In [1]:
import import_ipynb
import torch
import dataloader
import generate_mahalanobis
import regression_mahalanobis
import generate_odin
import calmetric
from models.resnet import ResNet34
from models.densenet import DenseNet3
import os
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

: 

In [ ]:
id_datasets = ['cifar10', 'cifar100']
ood_dataset = 'svhn'
model_names = ['resnet34', 'densenet3']
batch_size = 128
magnitude = 0.0014
temperature = 1000

for id_dataset in id_datasets:
    if id_dataset == 'cifar10':
        num_classes = 10
    elif id_dataset == 'cifar100':
        num_classes = 100
        
    for model_name in model_names:
        if model_name == 'resnet34':
            model = ResNet34(num_c=num_classes).to(device)
        elif model_name == 'densenet3':
            model = DenseNet3(100, num_classes, growth_rate=12).to(device)
            
        model_path = f'./pretrained/{model_name}_{id_dataset}.pth'
        model.load_state_dict(torch.load(model_path, weights_only=True))
        model.eval()
        
        mean, std = dataloader.get_mean_std(id_dataset)    
        
        id_trainloader, id_testloader = dataloader.get_imageloader(id_dataset, batch_size, mean, std)    
        _, out_testloader = dataloader.get_imageloader(ood_dataset, batch_size, mean, std)    
        
        ##### ODIN #####
        
        file_path = f'./softmax_scores/{model_name}_{id_dataset}'
        
        if not os.path.exists(file_path):
            os.makedirs(file_path)
            
        generate_odin.odin(model, id_testloader, out_testloader, magnitude, temperature, std, file_path)
        calmetric.metric(model_name, id_dataset, ood_dataset, file_path)
        
        ##### Mahalanobis #####
            
        file_path = f'./output/{model_name}_{id_dataset}'
        
        if not os.path.exists(file_path):
            os.makedirs(file_path)
        
        # generate_mahalanobis.mahalanobis(model, id_trainloader, id_testloader, out_testloader, num_classes, magnitude, std, file_path)
        # regression_mahalanobis.regression(id_dataset, ood_dataset, file_path, score=f'Mahalanobis_{str(magnitude)}')
        

num_samples: 10000
Processing in-distribution images
Processing out-of-distribution images
Neural network architecture:                 resnet34
In-distribution dataset:                      cifar10
Out-of-distribution dataset:                     svhn

                              ODIN
FPR at TPR 95%:              39.3%
Detection error:             15.8%
AUROC:                       90.3%
AUPR In:                     88.9%
AUPR Out:                    90.5%
num_samples: 10000
Processing in-distribution images
Processing out-of-distribution images
Neural network architecture:                densenet3
In-distribution dataset:                      cifar10
Out-of-distribution dataset:                     svhn

                              ODIN
FPR at TPR 95%:              54.4%
Detection error:             18.7%
AUROC:                       88.6%
AUPR In:                     89.1%
AUPR Out:                    87.1%
num_samples: 10000
Processing in-distribution images
